In [2]:
import io
import os
import math
import stackstac
import geojson
import dask_gateway
import planetary_computer
import rasterio.features
import azure.storage.blob
import numpy as np
import xarray as xr
import rioxarray as rioxr
import pysptools.abundance_maps as amp
import matplotlib.pyplot as plt
from dask.distributed import PipInstall, Lock
from scipy.stats import mode
from dask_gateway import GatewayCluster
from pystac_client import Client
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

In [3]:
def install():
    os.system('pip install pysptools') 
    os.system('pip install cvxopt')  

def read_file(file):
    with open(file) as f:
        content = f.read()
    return content

def load_blob_grid(blob_client):
    return geojson.loads(blob_client.download_blob().readall())

def get_tile(grid, h, v):
    return [x for x in grid['features'] if x['properties']['horizontal'] == h
            and x['properties']['vertical'] == v][0]['geometry']

def get_bbox(geometry):
    return rasterio.features.bounds(tile)

def get_container(container, connection_string):
    container_client = azure.storage.blob.ContainerClient.from_connection_string(
        connection_string, container_name=container
    )
    return container_client

def get_blob(container_client, blob_name):
    blob_client = container_client.get_blob_client(blob_name)
    return blob_client

def get_cluster(n=20, ncore=8, memory=16):
    gateway = dask_gateway.Gateway()
    cluster_options = gateway.cluster_options()
    cluster_options["worker_cores"] = ncore
    cluster_options["worker_memory"] = memory
    
    cluster = gateway.new_cluster(cluster_options)
    cluster.scale(n)
    client = cluster.get_client()
    return (cluster, client)

def register_package():
    plugin = PipInstall(packages=['pysptools', 'cvxopt'], pip_options=['--upgrade'])
    client.register_worker_plugin(plugin)

endmembers = np.array([[500, 900, 400, 6100, 3000, 1000],
                       [1400, 1700, 2200, 3000, 5500, 3000],
                       [2000, 3000, 3400, 5800, 6000, 5800],
                       [0, 0, 0, 0, 0, 0],
                       [9000, 9600, 8000, 7800, 7200, 6500]], dtype=np.int16)

In [4]:
catalog = Client.open('https://planetarycomputer.microsoft.com/api/stac/v1')
#catalog = Client.open('https://planetarycomputer-staging.microsoft.com/api/stac/v1')

def get_epsg(items):
    epsgs = [x.properties['proj:epsg'] for x in items]
    return 'EPSG: ' + str(mode(epsgs).mode[0])

def search_landsat_images(start, end, geometry, limit=1000):
    search = catalog.search(
        intersects = geometry,
        datetime = start + '/' + end,
        collections = ['landsat-c2-l2'],
        limit = 1000,
        query={'landsat:collection_category': {'eq': 'T1'}, 
               'eo:cloud_cover': {'lt': 90}}
    )
    return list(search.get_items())

def get_landsat_stack(start, end, geometry, chunksize=1024):
    items = search_landsat_images(start, end, geometry)
    signed_items = [planetary_computer.sign(item).to_dict() for item in items]
    
    bbox = get_bbox(geometry)
    epsg = get_epsg(items)
    
    data = (
        stackstac.stack(
            signed_items,
            assets=['blue', 'green', 'red', 'nir08', 'swir16', 'swir22', 'qa_pixel'],
            #assets=['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL'],
            chunksize=(1, 1, chunksize, chunksize),
            resolution=30,
            epsg=epsg,
            bounds_latlon=bbox
        )
        .assign_coords(band=['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2', 'QA'])
    )
    return data#.chunk((-1, -1, chunksize, chunksize))

def mask_landsat(col):
    good = [21824, 21952, 5440, 5504]
    mask = col.sel(band='QA').astype('uint16').isin(good)
    return (col.sel(band=['Red', 'Green', 'Blue', 'NIR', 'SWIR1', 'SWIR2'])
            .where(mask==1))

def to_surface_reflectance(col):
    return ((col * 0.0000275 - 0.2) * 10000) 

def array_to_frac_year(array, days_in_year=365.25):
    return array.time.dt.year + array.time.dt.day / days_in_year

def construct_dependents(array, days_in_year=365.25):
    x1 = array_to_frac_year(array, days_in_year)
    omega = 2 * math.pi
    x2 = np.cos(x1 * omega)
    x3 = np.sin(x1 * omega)
    return (
        xr.concat([x1, x2, x3], dim='x')
        .assign_coords(x=['x1', 'x2', 'x3'])
        .transpose(*('time', 'x'))
    )

def xr_unmix(col, endmembers):
    def unmix(M, U):
        M2 = M.squeeze()
        mask = (~np.isnan(M2)).min(axis=-1)
        M3 = M2.astype('int16')
        FCLS = amp.FCLS()
        unmixed = FCLS.map(M3, U, mask=mask)
        unmixed[mask==0, :] = np.nan
        return np.expand_dims(unmixed, axis=0)
    
    col2 = col.chunk((1, -1, -1, -1))
    return (
        xr.apply_ufunc(
            unmix, col2,
            input_core_dims=[['x', 'y', 'band']], 
            output_core_dims=[['x', 'y', 'lsma']],
            exclude_dims=set(('band',)), 
            kwargs={'U': endmembers},
            dask='parallelized', 
            output_dtypes=[col.dtype],
            output_sizes={'lsma': 5}
        )
        .rename({'lsma': 'band'})
        .assign_coords(band=['gv','npv','soil','shade','cloud'])
    )

def calculate_ndfi(col, scale=10000, chunksize=512):
    col2 = col.chunk((1, chunksize, chunksize, 1))
    
    gv = col2.sel(band='gv')
    npv = col2.sel(band='npv')
    soil = col2.sel(band='soil')
    shade = col2.sel(band='shade')
    cloud = col2.sel(band='cloud')

    gv_frac = (gv / (1 - shade)) + (npv + soil)
    mask = ((cloud < 0.2) & (shade < 1) & (gv_frac > 0)).astype('uint16')
    ndfi = (gv / (1 - shade) - (npv + soil)) / gv_frac * scale
    return ndfi.where(mask==1)

def xr_model_fit(col, chunksize=128):
    def model_fit(Y, X):
        y_true = Y[~np.isnan(Y)]
        if (y_true.size == 0):
            return np.array(
                [0, 0, 0, 0, 0], 
                ndmin=2, 
                dtype='float64'
            )
        else:
            regr = linear_model.LinearRegression()
            x_true = X[~np.isnan(Y), :]
            lm = regr.fit(x_true, y_true)
            coef = lm.coef_
            intercept = lm.intercept_
            y_pred = lm.predict(x_true)
            rmse = mean_squared_error(y_true=y_true, y_pred=y_pred, squared=False)
            return np.array(
                [intercept, coef[0], coef[1], coef[2], rmse], 
                ndmin=2, 
                dtype='float64'
            )
    
    X = construct_dependents(col)
    col2 = col.chunk((-1, chunksize, chunksize))
    return (
        xr.apply_ufunc(
            model_fit, col2,
            input_core_dims=[['time']], 
            output_core_dims=[['fit']],
            exclude_dims=set(('time',)), 
            kwargs={'X': X},
            dask='parallelized', 
            vectorize=True,
            output_dtypes=[col.dtype],
            output_sizes={'fit': 5}
        )
        .rename({'fit': 'band'})
        .assign_coords(band=['incpt','slope','cos','sin','rmse'])
        .transpose(*('band', 'y', 'x'))
    )

def export_to_drive(img, des, driver='COG', nodata=0, dask=False, client=None):
    dataset = (img
               .to_dataset(dim='band')
               .rio.write_crs(img.coords['epsg'].item())
              )
    
    for data_var in dataset.data_vars:
        dataset[data_var].rio.write_nodata(nodata, inplace=True)
    
    if dask:
        dataset.rio.to_raster(des, driver=driver, tiled=True, lock=Lock('fnrtm', client=client))
    else:
        dataset.rio.to_raster(des, driver=driver)
    
def export_to_blob(img, container_client, blob, driver='COG', nodata=0, dask=False, client=None):
    dataset = (img
               .to_dataset(dim='band')
               .rio.write_crs(img.coords['epsg'].item())
              )
    
    for data_var in dataset.data_vars:
        dataset[data_var].rio.write_nodata(nodata, inplace=True)
    
    with io.BytesIO() as buffer:
        if dask:
            dataset.rio.to_raster(buffer, driver=driver, tiled=True, lock=Lock('fnrtm', client=client))
        else:
            dataset.rio.to_raster(buffer, driver=driver)
        buffer.seek(0)
        blob_client = container_client.get_blob_client(blob)
        blob_client.upload_blob(buffer, overwrite=True)

In [5]:
connection_string = read_file('/home/jovyan/fnrtm/files/connect.txt')
container_client = get_container('misc', connection_string)
blob_client = get_blob(container_client, 'amazon_grid.geojson')
training_container = get_container('training', connection_string)

In [6]:
grid_blob = load_blob_grid(blob_client)
tile = get_tile(grid_blob, 41, 28)

In [7]:
(cluster, client) = get_cluster(20, 4, 32)
print(cluster.dashboard_link)

https://pccompute.westeurope.cloudapp.azure.com/compute/services/dask-gateway/clusters/prod.7778940264bf4bf78c83dfaebf8bc984/status


In [8]:
lst = get_landsat_stack('2019-01-01', '2021-12-31', tile, 1024)#[:, :, 0:200, 0:200]

/tmp/ipykernel_1042/2285019786.py:6: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  return 'EPSG: ' + str(mode(epsgs).mode[0])
/srv/conda/envs/notebook/lib/python3.10/site-packages/stackstac/prepare.py:363: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


In [10]:
processed = to_surface_reflectance(mask_landsat(lst))

In [11]:
processed

<xarray.DataArray 'stackstac-0230bff97d586727f3c59078a7a0bc65' (time: 364,
                                                                band: 6,
                                                                y: 5169, x: 5145)>
dask.array<mul, shape=(364, 6, 5169, 5145), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/24)
  * time                         (time) datetime64[ns] 2019-01-14T14:32:54.72...
    id                           (time) <U31 'LC08_L2SP_001066_20190114_02_T1...
  * band                         (band) <U5 'Red' 'Green' ... 'SWIR1' 'SWIR2'
  * x                            (x) float64 7.183e+05 7.183e+05 ... 8.726e+05
  * y                            (y) float64 -9.483e+05 ... -1.103e+06
    eo:cloud_cover               (time) float64 70.31 56.48 66.0 ... 82.95 87.37
    ...                           ...
    landsat:collection_number    <U2 '02'
    view:off_nadir               int64 0
    landsat:scene_id             (time) <U21 'LC80010662019014LGN00' ... 'LC8...
    landsat:wrs_path             (time) <U3 '001' '001' '233' ... '233' '233'
    instruments                  (time) object ['oli', 'tirs'] ... ['oli', 't...
    epsg                         <U11 'EPSG: 32619'

2023-05-10 18:48:06,658 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


In [23]:
unmixed = xr_unmix(processed, endmembers)

/tmp/ipykernel_982/2285019786.py:75: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  xr.apply_ufunc(


In [24]:
ndfi = calculate_ndfi(unmixed, 512)

In [25]:
fit = xr_model_fit(ndfi, 128)

/tmp/ipykernel_982/2285019786.py:129: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  xr.apply_ufunc(


In [26]:
register_package()

In [27]:
#export_to_drive(fit, '/home/jovyan/fnrtm/data/training/test.tif', dask=False)
export_to_blob(fit, training_container, 'FNRT_041028_1921.tif', dask=False)

ERROR 4: `/vsimem/200a2486-0526-42c2-a4b8-cd78695fcb47/200a2486-0526-42c2-a4b8-cd78695fcb47.tif' not recognized as a supported file format.


KeyboardInterrupt: 

In [ ]:
cluster.close()